### ScratchPad for Bug
 **LangLearner Activation Stats**

https://github.com/fastai/fastai/issues/2850#issuecomment-710082711

In [1]:
import torch 
import torch.nn as nn
from fastai.text.all import *

def hook_fn(m, i, o):
  print(m._get_name())
  print("     HOOK WORKING!    ")

model =  AWD_LSTM(3, 5, 6, 2)

In [2]:
flat_model =  flatten_model(model)
layer_1    =  flat_model[0]

inp_data   = torch.randint(3, (64,72))

In [3]:
out = layer_1(inp_data)

In [5]:
layer_1.register_forward_hook(hook_fn)

In [6]:
out = layer_1(inp_data)

Embedding
     HOOK WORKING!    


### New Example

In [54]:
path = untar_data(URLs.IMDB_SAMPLE)
imdb = pd.read_csv(path/'texts.csv')

In [55]:
N = 40
imdb_sample = imdb.iloc[:N]

In [64]:
imdb_lm = TextDataLoaders.from_df(imdb_sample, text_col='text', 
                                  is_lm=True, bs=10)

learn   = language_model_learner(imdb_lm, AWD_LSTM, 
                                 cbs = [ActivationStats],
                                 metrics=[accuracy]);

/home/user/anaconda3/envs/devfastai/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [68]:
imdb_lm.num_workers

1

In [63]:
learn.model

SequentialRNN(
  (0): AWD_LSTM(
    (encoder): Embedding(512, 400, padding_idx=1)
    (encoder_dp): EmbeddingDropout(
      (emb): Embedding(512, 400, padding_idx=1)
    )
    (rnns): ModuleList(
      (0): WeightDropout(
        (module): LSTM(400, 1152, batch_first=True)
      )
      (1): WeightDropout(
        (module): LSTM(1152, 1152, batch_first=True)
      )
      (2): WeightDropout(
        (module): LSTM(1152, 400, batch_first=True)
      )
    )
    (input_dp): RNNDropout()
    (hidden_dps): ModuleList(
      (0): RNNDropout()
      (1): RNNDropout()
      (2): RNNDropout()
    )
  )
  (1): LinearDecoder(
    (decoder): Linear(in_features=400, out_features=512, bias=True)
    (output_dp): RNNDropout()
  )
)

In [57]:
learn.fit(1)

epoch,train_loss,valid_loss,accuracy,time
0,3.867580,3.569041,0.287460,00:15


In [58]:
my_stats = learn.activation_stats.stats[0]

In [59]:
list(my_stats)

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'mean': 0.6898623704910278,
  'std': 2.4118845462799072,
  'near_zero': 0.4226209852430556}]

In [60]:
len(my_stats)

9

In [61]:
[stat for stat in my_stats]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'mean': 0.6898623704910278,
  'std': 2.4118845462799072,
  'near_zero': 0.4226209852430556}]

### Setup Vision

In [40]:
from fastai.vision.all import *

path = untar_data(URLs.MNIST_TINY)

df = pd.read_csv(path / 'labels.csv')
df.head()

,name,label
0,train/3/7463.png,3
1,train/3/9829.png,3
2,train/3/7881.png,3
3,train/3/8065.png,3
4,train/3/7046.png,3


In [41]:
train_mask = df['name'].map(lambda e: 'train' in e)
df         = df[train_mask]
df.shape

(709, 2)

In [42]:
db = DataBlock(blocks=(ImageBlock, CategoryBlock),
              get_y=ColReader('label'),
              get_x=ColReader('name', pref=path), 
         )

dls = db.dataloaders(df, path=path)

In [43]:
x, y = dls.one_batch()

x.shape

torch.Size([64, 3, 28, 28])

In [45]:
learn = cnn_learner(dls, resnet18,cbs=[ActivationStats])
learn.fit(1)

epoch,train_loss,valid_loss,time
0,0.929141,0.272355,00:05


### Process ActivationStats

In [47]:
AS = learn.activation_stats

In [48]:
type(AS)

fastai.callback.hook.ActivationStats

In [50]:
len(AS.stats)

8

In [53]:
list(AS.stats[-1])

[{'mean': -0.016696732491254807,
  'std': 3.9601614475250244,
  'near_zero': 0.5888397839604592},
 {'mean': 0.18121559917926788,
  'std': 0.4113817811012268,
  'near_zero': 0.33504688496492346},
 {'mean': -1.0923711061477661,
  'std': 1.3479244709014893,
  'near_zero': 0.8343231823979592},
 {'mean': -0.03415536507964134,
  'std': 0.41720473766326904,
  'near_zero': 0.5016691246811225},
 {'mean': -0.06179334595799446,
  'std': 0.40240198373794556,
  'near_zero': 0.6453832509566326},
 {'mean': 0.0030065076425671577,
  'std': 0.40686172246932983,
  'near_zero': 0.5407914142219388},
 {'mean': -0.6589341163635254,
  'std': 1.2419774532318115,
  'near_zero': 0.724968112244898},
 {'mean': -0.08352046459913254,
  'std': 0.37421756982803345,
  'near_zero': 0.5910694355867347},
 {'mean': -0.04646537825465202,
  'std': 0.30503302812576294,
  'near_zero': 0.6286023198341837},
 {'mean': -0.03036130778491497,
  'std': 0.4497096538543701,
  'near_zero': 0.5694256218112245},
 {'mean': -0.2516322135925

In [52]:
list(AS.stats[0])

[{'mean': -0.016670122742652893,
  'std': 3.937549114227295,
  'near_zero': 0.5889157665019132},
 {'mean': 0.18112032115459442,
  'std': 0.4113897979259491,
  'near_zero': 0.33511913066007654},
 {'mean': -1.0923278331756592,
  'std': 1.3463300466537476,
  'near_zero': 0.8343281648596939},
 {'mean': -0.0341365709900856,
  'std': 0.4170074760913849,
  'near_zero': 0.5035076530612245},
 {'mean': -0.06342763453722,
  'std': 0.403425931930542,
  'near_zero': 0.6463349011479592},
 {'mean': 0.003462872002273798,
  'std': 0.40673619508743286,
  'near_zero': 0.5397401147959183},
 {'mean': -0.6608835458755493,
  'std': 1.2439695596694946,
  'near_zero': 0.7248883928571429},
 {'mean': -0.08357392251491547,
  'std': 0.37405723333358765,
  'near_zero': 0.5912637515943877},
 {'mean': -0.04780922085046768,
  'std': 0.3034406006336212,
  'near_zero': 0.6304557955994898},
 {'mean': -0.029983943328261375,
  'std': 0.4499497413635254,
  'near_zero': 0.5678162866709183},
 {'mean': -0.25282856822013855,
  

### Replicate in Vision
